In [53]:
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

 # This thing should be tried
        # for i in range(x.size(1)):
        #     if tensor is None:
        #         tensor = self.layer(x[:, i:i+1, :, :])
        #     else:
        #         tensor = torch.cat((tensor, self.layer(x[:, i:i+1, :, :])), 1)

# Ideas
- conv + fc >>> add all channels
- conv + add all channels >>> fc

In [54]:
class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.name = 'Model_1'
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=5, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(6*44*44, 1)

    def forward(self, x):
        tensor = None
        for i in range(x.size(1)):
            if tensor is None:
                tensor = self.layer(x[:, i:i+1, :, :])
            else:
                tensor = tensor + self.layer(x[:, i:i+1, :, :])
        
        tensor = self.flatten(tensor)
        tensor = self.fc(tensor)
        tensor = torch.sigmoid(tensor)
        return tensor

In [55]:
model = Model_1()
model

Model_1(
  (layer): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(5, 5))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=11616, out_features=1, bias=True)
)

In [56]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 44, 44]             156
       BatchNorm2d-2            [-1, 6, 44, 44]              12
              ReLU-3            [-1, 6, 44, 44]               0
            Conv2d-4            [-1, 6, 44, 44]             156
       BatchNorm2d-5            [-1, 6, 44, 44]              12
              ReLU-6            [-1, 6, 44, 44]               0
            Conv2d-7            [-1, 6, 44, 44]             156
       BatchNorm2d-8            [-1, 6, 44, 44]              12
              ReLU-9            [-1, 6, 44, 44]               0
          Flatten-10                [-1, 11616]               0
           Linear-11                    [-1, 1]          11,617
Total params: 12,121
Trainable params: 12,121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/ba

In [57]:
writer = SummaryWriter(log_dir=f'runs/{model.name}')
writer.add_graph(model, torch.rand(1, 3, 224, 224))